In [1]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

import pytorch_lightning as plpl

import torch

from helper import load_dataset
from model import Data, SoftMaxLit, TransformerModel, get_dataloaders, LogisticRegression

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.device_count()

1

In [4]:
device = torch.cuda.current_device()

In [5]:
device

0

In [6]:
BATCH_SIZE = 4
NUM_EPOCH = 4
DEV = True

# Because DEV
df = load_dataset('../dataset/training.json', test=True)[:100]

model_names = ['albert', 'electra', 'roberta', 'xlnet']

In [7]:
for cur_model_name in model_names:
    TransformerModel(cur_model_name).dataset(df, DEV, save=True, delete=True)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|█████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.71s/it]
Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_pr

In [8]:
checkpoints = []
for cur_model_name in model_names:
    cur_model_name
    cur_dataset_x = torch.load(f'pretrained--dev=True--model={cur_model_name}.pt')
    cur_data = Data(df, x=cur_dataset_x)
    cur_dataloaders = get_dataloaders(cur_data, BATCH_SIZE)
    cur_model = SoftMaxLit(TransformerModel.MODELS[cur_model_name]['dim'], 2)
    checkpoint_callback = ModelCheckpoint(
        save_top_k=1,
        monitor='val_loss',
        mode='min',
        filename=f'model={cur_model_name}--dev={DEV}' + '--{epoch}-{step}--{val_loss:.2f}'
    )

    trainer = pl.Trainer(callbacks=[checkpoint_callback], max_epochs=NUM_EPOCH)
    trainer.fit(model=cur_model, train_dataloaders=cur_dataloaders['train'], val_dataloaders=cur_dataloaders['val'])
    trainer.test(cur_model, dataloaders=cur_dataloaders['test'])
    checkpoints.append(checkpoint_callback.best_model_path)

    del cur_dataset_x
    del cur_data.x
    torch.cuda.empty_cache()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2023-09-16 20:14:57.203127: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-16 20:14:57.234589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To 

Sanity Checking: 0it [00:00, ?it/s]

/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/duke/.local/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machi

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=4` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8999999761581421     │
└───────────────────────────┴───────────────────────────┘

In [9]:
pretrained_datasets_x = [
    "pretrained--dev=True--model=albert.pt",
    "pretrained--dev=True--model=electra.pt",
    "pretrained--dev=True--model=roberta.pt",
    "pretrained--dev=True--model=xlnet.pt"
]

# checkpoints = [
#     'lightning_logs/version_0/checkpoints/model=albert--dev=True--epoch=2-step=60--val_loss=0.39.ckpt',
#     'lightning_logs/version_1/checkpoints/model=electra--dev=True--epoch=2-step=60--val_loss=0.53.ckpt',
#     'lightning_logs/version_4/checkpoints/model=roberta--dev=True--epoch=3-step=80--val_loss=0.63.ckpt',
#     'lightning_logs/version_5/checkpoints/model=xlnet--dev=True--epoch=3-step=80--val_loss=0.62.ckpt'
# ]

model_y_arr = []
for model_name, pretrained_dataset_x, ckpt in zip(model_names, pretrained_datasets_x, checkpoints):
    n_inputs = TransformerModel.MODELS[model_name]['dim']
    model = SoftMaxLit(n_inputs, 2).load_from_checkpoint(n_inputs=n_inputs, n_outputs=2, checkpoint_path=ckpt)
    x = torch.load(pretrained_dataset_x).to(device)
    y_hat = model(x)
    
    # Free up memory
    del x
    torch.cuda.empty_cache()
#     y_first = y_hat[:,0]
    y_first = y_hat
#     y_first = y_first.reshape((y_first.shape[0], 1))

    model_y_arr.append(y_first)

lr_dataset_x = torch.cat(model_y_arr, dim=1).detach()

In [10]:
NUM_EPOCH = 100

lr_dataset = Data(df, x=lr_dataset_x)
lr_dataloaders = get_dataloaders(lr_dataset, BATCH_SIZE)

# lr_model = LogisticRegression(input_dim=lr_dataset_x.shape[1], num_classes=2)
lr_model = SoftMaxLit(lr_dataset_x.shape[1], 2)

checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor='val_loss',
    mode='min',
    filename=f'model=lr--dev={DEV}'
)

trainer = pl.Trainer(callbacks = [checkpoint_callback], max_epochs=NUM_EPOCH) # callbacks=[checkpoint_callback]
trainer.fit(model=lr_model, train_dataloaders=lr_dataloaders['train'], val_dataloaders=lr_dataloaders['val'])
trainer.test(lr_model, dataloaders=lr_dataloaders['test'])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | linear    | Linear           | 18    
1 | softmax   | Softmax          | 0     
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
18        Trainable params
0         Non-trainable params
18        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │            1.0            │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 1.0}]

In [11]:
x = lr_dataset.x.to(device)
lr_model.to(device)
lr_model(x)

tensor([[0.2016, 0.7984],
        [0.0704, 0.9296],
        [0.0551, 0.9449],
        [0.0864, 0.9136],
        [0.9796, 0.0204],
        [0.9885, 0.0115],
        [0.0472, 0.9528],
        [0.1729, 0.8271],
        [0.0630, 0.9370],
        [0.0547, 0.9453],
        [0.0522, 0.9478],
        [0.3348, 0.6652],
        [0.0708, 0.9292],
        [0.9475, 0.0525],
        [0.0675, 0.9325],
        [0.0664, 0.9336],
        [0.9421, 0.0579],
        [0.9724, 0.0276],
        [0.0552, 0.9448],
        [0.0654, 0.9346],
        [0.7427, 0.2573],
        [0.0918, 0.9082],
        [0.3929, 0.6071],
        [0.9803, 0.0197],
        [0.9251, 0.0749],
        [0.9760, 0.0240],
        [0.0715, 0.9285],
        [0.0447, 0.9553],
        [0.9605, 0.0395],
        [0.1610, 0.8390],
        [0.6432, 0.3568],
        [0.8098, 0.1902],
        [0.9887, 0.0113],
        [0.9903, 0.0097],
        [0.1242, 0.8758],
        [0.0752, 0.9248],
        [0.0696, 0.9304],
        [0.0526, 0.9474],
        [0.0

In [12]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
